## Original Dataset Cleaning
#### Import libraries and dataset

In [1]:
import pandas as pd
import re
from src.cleanfuncs import parse_snake_case

In [2]:
books = pd.read_csv("../input/goodreads.csv")

#### Explore the data

In [3]:
books.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,# num_pages,ratings_count,text_reviews_count
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,439785960,9780439785969,eng,652,1944099,26249
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,439358078,9780439358071,eng,870,1996446,27613
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,439554934,9780439554930,eng,320,5629932,70390
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,439554896,9780439554893,eng,352,6267,272
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,043965548X,9780439655484,eng,435,2149872,33964


In [4]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13719 entries, 0 to 13718
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   bookID              13719 non-null  int64 
 1   title               13719 non-null  object
 2   authors             13719 non-null  object
 3   average_rating      13719 non-null  object
 4   isbn                13719 non-null  object
 5   isbn13              13719 non-null  object
 6   language_code       13719 non-null  object
 7   # num_pages         13719 non-null  object
 8   ratings_count       13719 non-null  int64 
 9   text_reviews_count  13719 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 1.0+ MB


#### Drop unnecessary columns

In [5]:
books["isbn"].value_counts().head() # isbn values are not unique

4            4
60096683     1
674705599    1
851703682    1
143038184    1
Name: isbn, dtype: int64

In [6]:
books.drop(columns=["bookID", "isbn"] , inplace=True)

#### Fix column dtypes

In [7]:
print(books.dtypes)

title                 object
authors               object
average_rating        object
isbn13                object
language_code         object
# num_pages           object
ratings_count          int64
text_reviews_count     int64
dtype: object


In [8]:
for col in ["average_rating", "isbn13", "# num_pages"]:
    books[col] = pd.to_numeric(books[col], errors="coerce") # "coerce" will return NaN for errors

In [9]:
books.isnull().sum()

title                 0
authors               0
average_rating        5
isbn13                1
language_code         0
# num_pages           5
ratings_count         0
text_reviews_count    0
dtype: int64

In [10]:
books.dropna(inplace=True)

In [11]:
# repeat to_numeric(), this time converting to integer types correctly as there is no NaN values
for col in ["average_rating", "isbn13", "# num_pages"]:
    downcast = "float" if col == "average_rating" else "unsigned"
    books[col] = pd.to_numeric(books[col], downcast=downcast)

In [12]:
books.dtypes

title                  object
authors                object
average_rating        float32
isbn13                 uint64
language_code          object
# num_pages            uint16
ratings_count           int64
text_reviews_count      int64
dtype: object

#### Set ISBN13 as dataframe index after checking uniqueness

In [13]:
books["isbn13"].unique().size == books.shape[0]

True

In [14]:
books.set_index("isbn13", inplace=True)
books.sort_index(inplace=True)
books.index.name = "ISBN"

#### Convert author column to list of authors

In [15]:
books["authors"] = books["authors"].str.split("-").str.join(", ")

#### Convert different english speaking country language codes to "eng"

In [16]:
books["language_code"].value_counts()

eng      10594
en-US     1699
spa        419
en-GB      341
ger        238
fre        209
jpn         64
por         27
mul         21
ita         19
zho         16
grc         12
en-CA        9
rus          7
nl           7
swe          6
glg          4
tur          3
enm          3
cat          3
lat          3
ara          2
srp          1
ale          1
msa          1
heb          1
dan          1
nor          1
wel          1
gla          1
Name: language_code, dtype: int64

In [17]:
books["language_code"] = books["language_code"].str.replace(r"en-\w{2}", "eng")

#### Change column names to prettier format

In [18]:
books.columns = [parse_snake_case(col) for col in books.columns]
books.index.name = books.index.name.upper()  

#### Show final dataframe and export to csv

In [19]:
books.head()

,Title,Authors,Average Rating,Language Code,Num Pages,Ratings Count,Text Reviews Count
ISBN,,,,,,,
8987059752,The World's First Love: Mary Mother of God,Fulton J. Sheen,4.58,eng,276,571,55
20049130001,The Illuminati,Larry Burkett,3.71,eng,352,59,8
23755004321,The Servant Leader,Kenneth H. Blanchard,4.07,eng,128,256,24
34406054602,What Life Was Like in the Jewel in the Crown: ...,"Time, Life Books",3.64,eng,168,42,4
49086007763,Cliffs Notes on Aristophanes' Lysistrata The ...,W. John Campbell,2.33,eng,80,3,0


In [20]:
books.to_csv("../output/books_clean.csv")

#### Search dataset for data science books

In [21]:
books[books["Title"].str.contains(r"python", flags=re.IGNORECASE)]

,Title,Authors,Average Rating,Language Code,Num Pages,Ratings Count,Text Reviews Count
ISBN,,,,,,,
9780312311452,The Pythons Autobiography by The Pythons,"Graham Chapman, John Cleese, Terry Gilliam, Er...",4.08,eng,368,73,11
9780679726470,The Complete Monty Python's Flying Circus: All...,"Graham Chapman, Eric Idle, Terry Gilliam, John...",4.38,eng,352,3129,38
9780679726487,The Complete Monty Python's Flying Circus: All...,"Graham Chapman, John Cleese, Terry Gilliam, Er...",4.44,eng,384,1191,18
9780752852423,The Pythons Autobiography by The Pythons,"Michael Palin, Terry Gilliam, Graham Chapman",4.08,eng,359,17,1


In [22]:
books[books["Title"].str.contains(r"data science", flags=re.IGNORECASE)]

,Title,Authors,Average Rating,Language Code,Num Pages,Ratings Count,Text Reviews Count
ISBN,,,,,,,


In [23]:
books[books["Title"].str.contains(r"data analysis", flags=re.IGNORECASE)]

,Title,Authors,Average Rating,Language Code,Num Pages,Ratings Count,Text Reviews Count
ISBN,,,,,,,


In [24]:
books[books["Title"].str.contains(r"machine learning", flags=re.IGNORECASE)]

,Title,Authors,Average Rating,Language Code,Num Pages,Ratings Count,Text Reviews Count
ISBN,,,,,,,
